In [145]:
from tqdm import tqdm, trange
import json

l = []
with open(f"ToMi_processed_data/test.txt", 'r') as f_in:
    for line in f_in:
        l.append(json.loads(line))

In [146]:
print(l[0]['context'])

1 Jackson entered the hall.
2 Chloe entered the hall.
3 The boots is in the bathtub.
4 Jackson exited the hall.
5 Jackson entered the dining_room.
6 Chloe moved the boots to the pantry.


In [147]:
l[0]

{'context': '1 Jackson entered the hall.\n2 Chloe entered the hall.\n3 The boots is in the bathtub.\n4 Jackson exited the hall.\n5 Jackson entered the dining_room.\n6 Chloe moved the boots to the pantry.',
 'question': '7 Where was the boots at the beginning?',
 'label': 'bathtub 1'}

In [148]:
import pandas as pd

df = pd.DataFrame({
"context":[],
"question":[],
"label":[]
})

for i in trange(len(l)):
    df.loc[len(df)] = [l[i]['context'], l[i]['question'], l[i]['label']]

100%|████████████████████████████████████████████████████████████████████████████████████| 5994/5994 [00:06<00:00, 957.75it/s]


In [149]:
df['context'].nunique()

999

In [150]:
print(df.iloc[0]['context'], "\n\nQuestions:")
print("\n".join(   [" ".join(i.split(" ")[1:]) for i in df.iloc[0:6]['question'].to_list()]  ))

1 Jackson entered the hall.
2 Chloe entered the hall.
3 The boots is in the bathtub.
4 Jackson exited the hall.
5 Jackson entered the dining_room.
6 Chloe moved the boots to the pantry. 

Questions:
Where was the boots at the beginning?
Where will Chloe look for the boots?
Where does Chloe think that Jackson searches for the boots?
Where is the boots really?
Where will Jackson look for the boots?
Where does Jackson think that Chloe searches for the boots?


In [151]:
new = df.groupby('context').agg({'question': list, 'label': list}).reset_index()

In [154]:
li = []
with open("test.json", 'w') as f:
    for i in range(len(new)):
        li.append({
            "context": new.iloc[i]['context'],
            "question":new.iloc[i]['question'],
            "label":new.iloc[i]['label']
        })
    json.dump(li, f)

## Scoring

## Zeroshot

In [255]:
import json

def get_cost(items):
    
    count = {'prompt_tokens': 0,
      'completion_tokens': 0,
      'total_tokens': 0}

    for i in items:
        count['prompt_tokens'] = count['prompt_tokens'] + i['usage']['prompt_tokens']
        count['completion_tokens'] = count['completion_tokens'] + i['usage']['completion_tokens'] 
        count['total_tokens'] = count['total_tokens'] + i['usage']['total_tokens'] 

    dollars = 0.06 * (count['prompt_tokens']) / 1000  +  0.12 * (count['completion_tokens']) / 1000
    rupees = dollars * 83.3
    print(f"Total Cost: ${dollars} or Rs.{rupees}")
    
def get_average(l):
    return sum(l)/len(l)

In [299]:
import json 

setting = 'zeroshot'
with open(f"gpt4_pred_{setting}.json", 'r') as f:
    zeroshot_items = json.loads(f.read())

In [300]:
def extract_answer(item):
    
    final_score = []
    pred = json.loads(item['pred'])
    for q, a in zip(item['question'], item['label']):
        
        a = a.split(" ")[0]
        q = " ".join(q.split(" ")[1:])
        
        if pred[q].replace("in the", "").strip() == a:
            final_score.append(True)
        else:
            final_score.append(False)
            
        #print(q, "pred: ", pred[q], "\nLabel: ", a, pred[q] == a, "\n")
    return final_score

In [301]:
final_score = []
for item in zeroshot_items:
    final_score.extend(extract_answer(item))

print("Accuracy: ", get_average(final_score))
get_cost(zeroshot_items)

Accuracy:  0.6331331331331331
Total Cost: $21.0978 or Rs.1757.4467399999999


## Zeroshot-CoT

In [270]:
import json 

setting = 'zeroshotCoT'
with open(f"gpt4_pred_{setting}.json", 'r') as f:
    zeroshot_CoT_items = json.loads(f.read())

In [271]:
def extract_answer(item):
    return item['pred'].split("\n")[-1].split(":")[1].strip().replace(".", "") == item['label'].split(" ")[0]

In [272]:
scores = [extract_answer(item) for item in zeroshot_CoT_items]
print("Accuracy: ", get_average(scores))
get_cost(zeroshot_CoT_items)

Accuracy:  0.7054166666666667
Total Cost: $40.68012 or Rs.3388.653996


## Fewshot

In [281]:
import json 

setting = 'fewshot'
with open(f"gpt4_pred_{setting}.json", 'r') as f:
    fewshot_items = json.loads(f.read())

In [282]:
def extract_answer(item):
    
    final_score = []
    pred = json.loads(item['pred'])
    for q, a in zip(item['question'], item['label']):
        
        a = a.split(" ")[0]
        q = " ".join(q.split(" ")[1:])
        
        if pred[q] == a:
            final_score.append(True)
        else:
            final_score.append(False)
            
        #print(q, "pred: ", pred[q], "\nLabel: ", a, pred[q] == a, "\n")
    return final_score

In [283]:
final_score = []
for item in fewshot_items:
    final_score.extend(extract_answer(item))

print("Accuracy: ", get_average(final_score))
get_cost(fewshot_items)

Accuracy:  0.7125458792125459
Total Cost: $32.84502 or Rs.2735.9901659999996


## Fewshot-CoT

In [296]:
import json 

setting = 'fewshotCoT'
with open(f"gpt4_pred_{setting}.json", 'r') as f:
    fewshot_CoT_items = json.loads(f.read())

In [297]:
def extract_answer(item):
    return item['pred'].split("\n")[-1].split(":")[1].strip().replace(".", "") == item['label'].split(" ")[0]

In [298]:
scores = [extract_answer(item) for item in fewshot_CoT_items]
print("Accuracy: ", get_average(scores))
get_cost(fewshot_CoT_items)

Accuracy:  0.8545833333333334
Total Cost: $128.11596 or Rs.10672.059468


In [307]:
10672 + 2735 + 3388 + 1757

18552

## CoT

In [87]:
import json

with open("test.json", 'r') as f:
    l = json.loads(f.read())

In [96]:
import random

random.seed(2023)
index = random.sample(range(1000), 400)

In [98]:
selected = []
for i in index:
    selected.append(l[i])

In [99]:
sum([len(i['question']) for i in selected])

2400

In [121]:
final = []
for item in selected:
    
    context = item['context']
    for q, a in zip(item['question'], item['label']):
        
        final.append({
        "context": context,
        "question": q,
        "label": a
        })

In [122]:
len(final)

2400

In [124]:
final[2]

{'context': '1 Evelyn entered the study.\n2 Avery entered the study.\n3 Benjamin entered the study.\n4 Benjamin loves the skirt\n5 The potato is in the container.\n6 Avery exited the study.\n7 Evelyn moved the potato to the treasure_chest.\n8 Evelyn exited the study.\n9 Avery entered the study.\n10 Benjamin loves the pineapple',
 'question': '11 Where does Evelyn think that Avery searches for the potato?',
 'label': 'container 1'}

In [125]:
with open('CoT_Test.json', 'w') as f:
    json.dump(final, f)

## running CoT 

In [162]:
with open("CoT_Test.json", 'r') as f:
    dataset = json.loads(f.read())
    
def get_zeroshot(item):
    
    context = item['context']
    prompt = context + "\n\nQuestion:\n" + item['question'] + "\n\nLet's think step by step\nFinally give answer in new line as Final_answer: your final answer(only specific key word asked in question)"
    
    return prompt, item['label']

In [157]:
for item in dataset:
    p, l = get_zeroshot(item)
    

In [158]:
print(p)

1 Evelyn entered the cellar.
2 Charlotte entered the cellar.
3 Lucas entered the cellar.
4 Evelyn exited the cellar.
5 The shirt is in the bottle.
6 Lucas moved the shirt to the cupboard.
7 Charlotte exited the cellar.
8 Lucas exited the cellar.
9 Charlotte entered the cellar.

Question:
10 Where does Charlotte think that Lucas searches for the shirt?

Let's think step by step
Finally give answer in new line as Final_answer: your final answer(only specific key word asked in question)


In [166]:
def get_zeroshot(item):
    
    context = item['context']
    prompt = context + "\n\nQuestion:\n" + item['question'] + "\n\nLet's think step by step\nFinally give answer in new line as Final_answer: your final answer(only specific key word asked in question)"
    
    return prompt, item['label']

def get_fewshot(item):
    
    prompt = """
[Context]:
1 Evelyn entered the cellar.
2 Charlotte entered the cellar.
3 Lucas entered the cellar.
4 Evelyn exited the cellar.
5 The shirt is in the bottle.
6 Lucas moved the shirt to the cupboard.
7 Charlotte exited the cellar.
8 Lucas exited the cellar.
9 Charlotte entered the cellar.

[Question]:
10 Where does Charlotte think that Lucas searches for the shirt?

Let's think step by step
Finally give answer in new line as Final_answer: your final answer(only specific key word asked in question)

[Answer]:
Step 1: Evelyn, Charlotte, and Lucas entered the cellar (points 1-3).
Step 2: Evelyn exited the cellar (point 4). At this moment, Charlotte and Lucas are still in the cellar.
Step 3: The shirt is in the bottle (point 5). It is not mentioned if Charlotte and Lucas saw the shirt being placed in the bottle, so we cannot assume they know its location.
Step 4: Lucas moved the shirt to the cupboard (point 6). Since Charlotte is in the cellar with Lucas, she witnesses him move the shirt to the cupboard.
Step 5: Charlotte exited the cellar (point 7). Now, Charlotte is no longer in the cellar, but she knows that Lucas moved the shirt to the cupboard.
Step 6: Lucas exited the cellar (point 8).
Step 7: Charlotte entered the cellar again (point 9).

Now, to answer the question "Where does Charlotte think that Lucas searches for the shirt?"

Based on the information provided, Charlotte last saw Lucas move the shirt to the cupboard. Therefore, she would believe that Lucas searches for the shirt in the cupboard.

Final_answer: cupboard.

[End of Examples]

"""
    
    context = item['context']
    prompt += "[Context]:\n" + context + "\n\n[Question]:\n" + item['question'] + """\n\nLet's think step by step
Finally give answer in new line as Final_answer: your final answer(only specific key word asked in question)\n\n[Answer]:"""
    
    return prompt, item['label']

In [168]:
p, la = get_fewshot(dataset[99])
print(p)


[Context]:
1 Evelyn entered the cellar.
2 Charlotte entered the cellar.
3 Lucas entered the cellar.
4 Evelyn exited the cellar.
5 The shirt is in the bottle.
6 Lucas moved the shirt to the cupboard.
7 Charlotte exited the cellar.
8 Lucas exited the cellar.
9 Charlotte entered the cellar.

[Question]:
10 Where does Charlotte think that Lucas searches for the shirt?

Let's think step by step
Finally give answer in new line as Final_answer: your final answer(only specific key word asked in question)

[Answer]:
Step 1: Evelyn, Charlotte, and Lucas entered the cellar (points 1-3).
Step 2: Evelyn exited the cellar (point 4). At this moment, Charlotte and Lucas are still in the cellar.
Step 3: The shirt is in the bottle (point 5). It is not mentioned if Charlotte and Lucas saw the shirt being placed in the bottle, so we cannot assume they know its location.
Step 4: Lucas moved the shirt to the cupboard (point 6). Since Charlotte is in the cellar with Lucas, she witnesses him move the shirt t

In [169]:
la

'box 1'

In [142]:
import openai

openai.api_type = ""
openai.api_base = ""
openai.api_version = ""
openai.api_key = ""


def gpt4(query, i, counter=0):
    
    try:
        messages = [
                       {"role": "system", "content": "You are a helpful AI assistant."},
                       {"role": "user", "content": query}
                   ]
        
        response = openai.ChatCompletion.create(
            engine="gpt-4-32k",  # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
            messages=messages,
            temperature=0
        )
        return response['choices'][0]['message']['content'], response['usage']
    
    except:
        if counter < 5:
            time.sleep(5)
            return gpt4(prompt, i, counter + 1)
        else:
            print("exception at:" + str(i))
            return "", {
    "completion_tokens": 0,
    "prompt_tokens": 0,
    "total_tokens": 0
}

In [148]:
p, l = get_zeroshot(dataset[45])
res, usage = gpt4(p, 0)

In [149]:
print(res, "\n\ngold:", l)

Step 1: Liam entered the kitchen.
Step 2: Olivia entered the kitchen.
Step 3: The persimmon is in the suitcase.
Step 4: Olivia exited the kitchen.
Step 5: Mila entered the back_yard.
Step 6: Mila exited the back_yard.
Step 7: Liam moved the persimmon to the bucket.

Final_answer: bucket 

gold: bucket 1
